### Analyse search terms on the e-commerce web server


#####  download the search term data set for the e-commerce web server and run analytic queries on it.


In [3]:
!pip install pyspark
!pip install findspark

In [58]:
import pandas as pd
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.ml.regression import LinearRegressionModel

In [5]:
sc = SparkContext()
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

23/06/15 06:11:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/15 06:11:37 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2023-06-15 06:11:49--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2023-06-15 06:11:49 (40.7 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [21]:
search_sdf = spark.read.csv("searchterms.csv", header=True, inferSchema=True)
search_sdf.show(3)

+---+-----+----+-------------+
|day|month|year|   searchterm|
+---+-----+----+-------------+
| 12|   11|2021|mobile 6 inch|
| 12|   11|2021|mobile latest|
| 12|   11|2021|  tablet wifi|
+---+-----+----+-------------+
only showing top 3 rows



In [27]:
num_rows = search_sdf.count()
num_cols = len(search_sdf.columns)

print("Number of rows: ", num_rows)
print("Number of columns: ", num_cols)

Number of rows:  10000
Number of columns:  4


In [28]:
search_sdf.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [29]:
search_sdf.dtypes

[('day', 'int'), ('month', 'int'), ('year', 'int'), ('searchterm', 'string')]

In [36]:
search_sdf.filter(search_sdf.searchterm.like("%gaming laptop%")).count()

499

In [52]:
(
search_sdf
 .groupBy("searchterm")
 .count()
 .orderBy(desc("count"))
 .show(5)
)

[Stage 35:==========================================>          (162 + 12) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [53]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xvf model.tar.gz

--2023-06-15 07:21:36--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-06-15 07:21:36 (8.27 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [63]:
model = LinearRegressionModel.load('sales_prediction.model')

In [ ]:
assembler = VectorAssembler(inputCols=["weight"],outputCol="features")
data = [[weight,0]]
columns = ["weight", "height"]
_ = spark.createDataFrame(data, columns)
__ = assembler.transform(_).select('features','height')
predictions = model.transform(__)
predictions.select('prediction').show()

In [ ]:
spark.stop()